In [1]:
import duckdb
import pandas as pd
from vanna.remote import VannaDefault

In [2]:
con = duckdb.connect("ads_warehouse.duckdb")

print(con.execute("SELECT * FROM ads_spend LIMIT 5").fetchdf())

        date platform account      campaign country   device    spend  clicks  \
0 2025-01-01     Meta   AcctA   Prospecting      MX  Desktop  1115.94     360   
1 2025-01-01   Google   AcctA  Brand_Search      CA   Mobile   789.43     566   
2 2025-01-01   Google   AcctA   Prospecting      BR  Desktop   381.40     133   
3 2025-01-01   Google   AcctC   Prospecting      US  Desktop  1268.34     891   
4 2025-01-01   Google   AcctA  Brand_Search      BR  Desktop  1229.70     628   

   impressions  conversions                  load_date source_file_name  
0        15840           29 2025-09-23 01:24:25.791371         File.csv  
1        22640           28 2025-09-23 01:24:25.791371         File.csv  
2        10241           12 2025-09-23 01:24:25.791371         File.csv  
3        49005           36 2025-09-23 01:24:25.791371         File.csv  
4        21352           31 2025-09-23 01:24:25.791371         File.csv  


In [3]:
from vanna.remote import VannaDefault

api_key = "ed4da24998bc4aab9cd997dff5676521"
model_name = "model_ads_spend"

vn = VannaDefault(model=model_name, api_key=api_key)


In [4]:
ddl = """
CREATE TABLE ads_spend (
    date DATE,
    platform TEXT,
    account TEXT,
    campaign TEXT,
    country TEXT,
    device TEXT,
    spend DOUBLE,
    clicks INTEGER,
    impressions INTEGER,
    conversions INTEGER
);
"""
vn.train(ddl=ddl)


Adding ddl: 
CREATE TABLE ads_spend (
    date DATE,
    platform TEXT,
    account TEXT,
    campaign TEXT,
    country TEXT,
    device TEXT,
    spend DOUBLE,
    clicks INTEGER,
    impressions INTEGER,
    conversions INTEGER
);



'18225206-ddl'

In [5]:
vn.train(documentation="CAC = spend / conversions. ROAS = (conversions * 100) / spend. Revenue = conversions * 100.")


vn.train(
    question="Obtener CAC promedio de los últimos 30 días.",
    sql="SELECT SUM(spend)/SUM(conversions) AS CAC FROM ads_spend WHERE date BETWEEN DATE '2025-06-01' AND DATE '2025-06-30'"
)

vn.train(
    question="Comparar ROAS últimos 30 días vs 30 días anteriores.",
    sql="""
WITH revenue AS (
  SELECT 
    CASE 
      WHEN date BETWEEN DATE '2025-06-01' AND DATE '2025-06-30' THEN 'last_30'
      WHEN date BETWEEN DATE '2025-05-02' AND DATE '2025-05-31' THEN 'prev_30'
    END AS period,
    SUM(conversions*100) AS revenue,
    SUM(spend) AS spend
  FROM ads_spend
  WHERE date BETWEEN DATE '2025-05-02' AND DATE '2025-06-30'
  GROUP BY period
)
SELECT period, revenue/spend AS roas FROM revenue;
"""
)


Adding documentation....


'4a2432de7e78a719ca1926aff0152433-sql'

In [6]:
question = "Compare CAC and ROAS for last 30 days vs prior 30 days."
sql_generated = vn.generate_sql(question)
print("SQL generado por Vanna:")
print(sql_generated)


SQL Prompt: [{'role': 'system', 'content': "You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE ads_spend (\n    date DATE,\n    platform TEXT,\n    account TEXT,\n    campaign TEXT,\n    country TEXT,\n    device TEXT,\n    spend DOUBLE,\n    clicks INTEGER,\n    impressions INTEGER,\n    conversions INTEGER\n);\n\n\n\n===Additional Context \n\nCAC = spend / conversions. ROAS = (conversions * 100) / spend. Revenue = conversions * 100.\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment 

In [7]:
result_df = con.execute(sql_generated).fetchdf()
print("Resultado de la consulta:")
print(result_df)


Resultado de la consulta:
    period        CAC      ROAS
0  prev_30  32.271528  3.098707
1  last_30  29.809261  3.354662
